In [28]:
import numpy as np
import os,cv2
import tensorflow as tf

#归一化数据范围到[-1, 1]
def normalized(data):
    mx = max(data)
    mn = min(data)
    return [(2 *(float(i) - mn) / (mx - mn)) - 1 for i in data]

class DataCooker(object):
    def __init__(self, image_path = os.path.join('.','data'), label_path = os.path.join('.','data','label.txt'), image_size = 256, batch_size = 1):
        #图像、标签路径
        self.image_path = image_path
        self.label_path = label_path
        self.image_size = image_size
        self.batch_size = batch_size
        #读取标签
        with open(self.label_path,'r') as f:
            lines = f.readlines()
        #记录标签对
        image_name_list = []
        label_list = []
        if len(lines) <= 0:
            print('Error: Do not have label data！')
        else:
            for line in lines:
                strs = line.strip().split()
                image_name_list.append(os.path.join(self.image_path,strs[0]))
                label_list.append(float(strs[1]))
        self.labels = np.array([image_name_list, normalized(label_list)])
        #batch未整除警告
        if len(self.labels[0])%batch_size != 0:
            print('Warning: Data Length(', len(self.image_list), ') is not divisible by Batch Size',batch_size, '!')
        
    def get_batch_data(self):
        #读取函数
        def __read_image_dataset(filename, label):
            image_string = tf.read_file(filename)
            image_decoded = tf.image.decode_jpeg(image_string)
            image_resized = tf.image.resize_images(image_decoded, (self.image_size, self.image_size))
            #image_normalized = image_resized/127.5 - 1
            return image_resized, label
        #读取图片        
        filenames = tf.constant(self.labels[0])
        labels= tf.constant(self.labels[1])
        dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
        self.dataset = dataset.map(__read_image_dataset)
        #设置batch
        self.batch_dataset = self.dataset.batch(self.batch_size)
        #创建迭代器
        self.iterator = self.batch_dataset.make_one_shot_iterator()
    
        return self.iterator.get_next()

if __name__ == '__main__':
    dc = DataCooker(os.path.join('/','data','Images'), os.path.join('/','data','labels.txt'), batch_size = 2)
    #print(dc.labels[0])
    img_iter = dc.get_batch_data()
    
    with tf.Session() as sess:
        res = sess.run(img_iter)
    cv2.imwrite('/data/1.jpg',cv2.cvtColor(res[0][0], cv2.COLOR_RGB2BGR))
        
        
        
        
        
        
        
        
        
        
        
        